In [1]:
wp_urls_text = '''
אלדד הדני

https://www.bh.org.il/he/%d7%90%d7%9c%d7%93%d7%93-%d7%94%d7%93%d7%a0%d7%99-eldad-ha-dani/

 

אבא מהרי

https://www.bh.org.il/he/%d7%90%d7%91%d7%90-%d7%9e%d7%94%d7%a8%d7%99/

 

יוסף הלוי

https://www.bh.org.il/he/%d7%99%d7%95%d7%a1%d7%a3-%d7%94%d7%9c%d7%95%d7%99/

 

זימנה ברהנה  

https://www.bh.org.il/he/%d7%96%d7%99%d7%9e%d7%a0%d7%94-%d7%91%d7%a8%d7%94%d7%a0%d7%94/

 

חזי עובדיה

https://www.bh.org.il/he/%d7%a2%d7%95%d7%91%d7%93%d7%99%d7%94-%d7%97%d7%96%d7%99/

 

בליינש זבדיה 

https://www.bh.org.il/he/%d7%91%d7%9c%d7%99%d7%99%d7%a0%d7%a9-%d7%96%d7%91%d7%93%d7%99%d7%94/

 

אברהם יצחק

https://www.bh.org.il/he/%d7%93%d7%a8-%d7%90%d7%91%d7%a8%d7%94%d7%9d-%d7%99%d7%a6%d7%97%d7%a7/

 

ייטאייש (טיטי) איינאו   

https://www.bh.org.il/he/%d7%99%d7%99%d7%98%d7%90%d7%99%d7%99%d7%a9-%d7%98%d7%99%d7%98%d7%99-%d7%90%d7%99%d7%99%d7%a0%d7%90%d7%95/
 
'''

In [5]:
wp_urls = [line.strip() for line in wp_urls_text.split("\n") if line.strip().startswith('http')]
wp_urls

['https://www.bh.org.il/he/%d7%90%d7%9c%d7%93%d7%93-%d7%94%d7%93%d7%a0%d7%99-eldad-ha-dani/',
 'https://www.bh.org.il/he/%d7%90%d7%91%d7%90-%d7%9e%d7%94%d7%a8%d7%99/',
 'https://www.bh.org.il/he/%d7%99%d7%95%d7%a1%d7%a3-%d7%94%d7%9c%d7%95%d7%99/',
 'https://www.bh.org.il/he/%d7%96%d7%99%d7%9e%d7%a0%d7%94-%d7%91%d7%a8%d7%94%d7%a0%d7%94/',
 'https://www.bh.org.il/he/%d7%a2%d7%95%d7%91%d7%93%d7%99%d7%94-%d7%97%d7%96%d7%99/',
 'https://www.bh.org.il/he/%d7%91%d7%9c%d7%99%d7%99%d7%a0%d7%a9-%d7%96%d7%91%d7%93%d7%99%d7%94/',
 'https://www.bh.org.il/he/%d7%93%d7%a8-%d7%90%d7%91%d7%a8%d7%94%d7%9d-%d7%99%d7%a6%d7%97%d7%a7/',
 'https://www.bh.org.il/he/%d7%99%d7%99%d7%98%d7%90%d7%99%d7%99%d7%a9-%d7%98%d7%99%d7%98%d7%99-%d7%90%d7%99%d7%99%d7%a0%d7%90%d7%95/']

In [45]:
import requests
from pyquery import PyQuery as pq
from dataflows import Flow, printer, dump_to_path

def get_items():
    for wp_url in wp_urls:
        res = requests.get(wp_url)
        doc = pq(res.content)
        assert doc.attr('lang') == 'he-IL', 'invalid lang: {}'.format(doc.attr('lang'))
        title = doc('h2.title').text()
        assert title and len(title) > 5, 'invalid title: {}'.format(title)
        description = '.'.join(doc('div.content-column p').text().split('.')[:4])
        assert len(description) > 30, 'invalid description: {}'.format(description)
        imgs = doc('div.content-column img')
        image_url = ('https:'+pq(imgs[0]).attr('src')) if imgs else ''
        yield {'url': wp_url, 'title': title, 'description': description, 'image_url': image_url}

Flow(
    get_items(),
    dump_to_path('.data/ethiopia-personalities'),
    printer(tablefmt='html', num_rows=9999)
).process()

#,url (string),title (string),description (string),image_url (string)
1,https://www.bh.org.il/he/%d7%90%d7%9c%d7%93%d7%93-%d7%94%d7%93%d7%a0%d7%99-eldad-ha-dani/,אלדד הדני,"אלדד הדני היה סוחר, נוסע יהודי ותלמיד חכם במאה ה-9. טען כי הוא בן שבט דן וסיפר ספורים ואגדות על עשרת ...",None
2,https://www.bh.org.il/he/%d7%90%d7%91%d7%90-%d7%9e%d7%94%d7%a8%d7%99/,אבא מהרי,"אבא מהרי היה נזיר יהודי, שחי ופעל באתיופיה במאה ה-19 ושימש כמנהיג הרוחני העליון של קהילת ביתא ישראל ...",None
3,https://www.bh.org.il/he/%d7%99%d7%95%d7%a1%d7%a3-%d7%94%d7%9c%d7%95%d7%99/,יוסף הלוי,"יוסף הלוי היה יהודי צרפתי שנולד בתורכיה. הוא היה מורה, חוקר, משורר ובלשן. היה פרופסור באוניברסיטת סו ...",https://www.bh.org.il/wp-content/uploads/Joseph_Halevy.jpg
4,https://www.bh.org.il/he/%d7%96%d7%99%d7%9e%d7%a0%d7%94-%d7%91%d7%a8%d7%94%d7%a0%d7%94/,זימנה ברהנה,"היה מחלוצי המאבק להעלאת קהילת ""ביתא ישראל"" למדינת ישראל ולפתיחת שערי הארץ בפניהם. הוא היה איש המוסד ...",https://www.bh.org.il/wp-content/uploads/זימנה_ברהני_באתיופיה_1992_אוסף_הרב_מנחם_ולדמן.jpg
5,https://www.bh.org.il/he/%d7%a2%d7%95%d7%91%d7%93%d7%99%d7%94-%d7%97%d7%96%d7%99/,חזי עובדיה,"חזי עובדיה היה רס""ר משמעת בצה""ל ופעיל מרכזי למען עליית ביתא ישראל.נולד ב-1922 בעיר אסמרה בירת אריתרי ...",None
6,https://www.bh.org.il/he/%d7%91%d7%9c%d7%99%d7%99%d7%a0%d7%a9-%d7%96%d7%91%d7%93%d7%99%d7%94/,בליינש זבדיה,בליינש נולדה באפריל 1967. דיפלומטית שכיהנה כשגרירת ישראל באתיופיה. מדליקת משואה ביום העצמאות כאשה פו ...,https://www.bh.org.il/wp-content/uploads/בליינש_זבדיה-1140x641.jpg
7,https://www.bh.org.il/he/%d7%93%d7%a8-%d7%90%d7%91%d7%a8%d7%94%d7%9d-%d7%99%d7%a6%d7%97%d7%a7/,"ד""ר אברהם יצחק","אברהם יצחק (נולד בשנת 1973) הוא קצין בצה""ל בדרגת אלוף-משנה ורופא, המכהן כמפקד רפואה פיקוד הדרום[1]. ...",None
8,https://www.bh.org.il/he/%d7%99%d7%99%d7%98%d7%90%d7%99%d7%99%d7%a9-%d7%98%d7%99%d7%98%d7%99-%d7%90% ...,ייטאייש (טיטי) איינאו,"נולדה ב-23 ביוני 1991. דוגמנית ומנחת טלוויזיה ישראלית, מלכת היופי של ישראל לשנת 2013. איינאו היא יוצ ...",https://www.bh.org.il/wp-content/uploads/Titi_Aynaw-913x1140.jpg


(<datapackage.package.Package at 0x7f54e6f4be50>,
 {'count_of_rows': 8,
  'bytes': 6797,
  'hash': '50d2cb20d4fb3d19e8dcec48583532de',
  'dataset_name': None})

In [1]:
import json
from urllib.parse import quote
from dataflows import Flow, load

def print_manual_items_json(row):
    item_json = '''    {
      "UnitType": 8,
      "UnitTypeDesc": "Personality",
      "Header": {
        "He": <<title>>,
        "En": ""
      },
      "UnitText1": {
        "He": <<description>>,
        "En": ""
      },
      "Slug": {
        "He": "",
        "En": ""
      },
      "video_url": null,
      "main_image_url": <<image_url>>,
      "preview_image_url": <<image_url>>,
      "image_urls": [<<image_urls>>],
      "item_url_he": <<url>>,
      "item_url_en": ""
    }, '''.replace(
        '<<title>>', json.dumps(row['title'], ensure_ascii=False)
    ).replace(
        '<<description>>', json.dumps(row['description'], ensure_ascii=False)
    ).replace(
        '<<image_url>>', json.dumps(quote(row['image_url']).replace('https%3A', 'https:')) if row['image_url'] else '""'
    ).replace(
        '<<image_urls>>', json.dumps(quote(row['image_url']).replace('https%3A', 'https:')) if row['image_url'] else ''
    ).replace(
        '<<url>>', json.dumps(row['url'])
    )
    print(item_json)

Flow(
    load('.data/ethiopia-personalities/datapackage.json'),
    print_manual_items_json
).process()

    {
      "UnitType": 8,
      "UnitTypeDesc": "Personality",
      "Header": {
        "He": "אלדד הדני",
        "En": ""
      },
      "UnitText1": {
        "He": "אלדד הדני היה סוחר, נוסע יהודי ותלמיד חכם במאה ה-9. טען כי הוא בן שבט דן וסיפר ספורים ואגדות על עשרת השבטים ועל \"בני משה\". סיפוריו קסמו ללב היהודים ועודדו אותם לשאוף לגאלה ולחיי חירות. אלדד הדני היה נוסע בין הקהילות היהודיות החשובות בבבל, צפון אפריקה וספרד",
        "En": ""
      },
      "Slug": {
        "He": "",
        "En": ""
      },
      "video_url": null,
      "main_image_url": "",
      "preview_image_url": "",
      "image_urls": [],
      "item_url_he": "https://www.bh.org.il/he/%d7%90%d7%9c%d7%93%d7%93-%d7%94%d7%93%d7%a0%d7%99-eldad-ha-dani/",
      "item_url_en": ""
    }, 
    {
      "UnitType": 8,
      "UnitTypeDesc": "Personality",
      "Header": {
        "He": "אבא מהרי",
        "En": ""
      },
      "UnitText1": {
        "He": "אבא מהרי היה נזיר יהודי, שחי ופעל באתיופיה במאה ה-19 ושימש

(<datapackage.package.Package at 0x7f8758fc03d0>, {})